# NOAA Real-Time Weather Data

## Set Up

### Import Packages

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import json # Reading jsons

# Database 

# import psycopg2
# from psycopg2 import sql

# Analysis

import numpy as np
# import arcpy
import pandas as pd

## API Pulls

### Forecast

In [72]:
def get_forecast(latlons):
    '''This function gets the forecast URL from National Weather Service's api
    and then gets the forecast for a list of lat/longs [(lat0,lon0),...]
    It also gets the current pressure for Minneapolis.
    
    It will return a dataframe of the forecasts with index aligning with the index of the latlons
    
    see here for API documentation:
    https://www.weather.gov/documentation/services-web-api
    '''

    # Initialize return value
    
    forecast_df = []

    # Define the headers for the request
    headers = {"Accept": "application/json"}
    
    # Get Local Current Pressure
    # Response is in Pascals, the conversion to millibars (PurpleAir Units) is 1/100
    
    pressure_response = requests.get('https://api.weather.gov/stations/KMSP/observations/latest',
                                     headers=headers)
    
    current_pressure = float(pressure_response.json()['properties']['barometricPressure']['value'])/100 
    
    # Define the URL to get gridpoints for the API endpoint
    gridpoint_url = "https://api.weather.gov/points/{lat},{lon}"
    
    for i, latlon in enumerate(latlons):
        
        lat = latlon[0]
        lon = latlon[1]
    
        # Send the request and get the response
        response = requests.get(gridpoint_url.format(lat=lat, lon=lon), headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            
            # Extract the forecast url from the response
            forecast_url = response.json()['properties']['forecast']

            forecast_response = requests.get(forecast_url, headers=headers)

            if forecast_response.status_code == 200:

                forecast_json = forecast_response.json()

                afternoon_df = pd.DataFrame(forecast_json['properties']['periods'][0])
                night_df = pd.DataFrame(forecast_json['properties']['periods'][1])

                full_df = pd.concat([afternoon_df, night_df]).loc['value'].reset_index()
                
                full_df['index'] = i
                full_df['pressure'] = current_pressure

                # Concatenate to the forecast_df
                
                if len(forecast_df) == 0:
                    
                    forecast_df = full_df
                    
                else:
                    
                    forecast_df = pd.concat([forecast_df, full_df], ignore_index = True)

        else:
            print("Request failed with status code {}".format(response.status_code))
            print("Error message: {}".format(response.text))

    return forecast_df

In [59]:
# Test with Latitude and longitude of Downtown Minneapolis, MN
lat = 44.9778
lon = -93.2650

latlons = [(lat,lon)]

get_forecast(latlons)

,index,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast,pressure
0,0,1,This Afternoon,2023-05-05T12:00:00-05:00,2023-05-05T18:00:00-05:00,True,66,F,None,50,10.555556,67,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,50...",Chance Showers And Thunderstorms,"A chance of rain showers before 1pm, then a ch...",10122.0
1,0,2,Tonight,2023-05-05T18:00:00-05:00,2023-05-06T06:00:00-05:00,False,56,F,None,70,11.666667,90,5 to 10 mph,E,"https://api.weather.gov/icons/land/night/tsra,...",Chance Showers And Thunderstorms then Showers ...,A chance of showers and thunderstorms before 8...,10122.0


In [61]:
# Try for all stations

cwd = os.getcwd()

data_path = os.path.join(cwd,'..', '..', 'data')

# Station Locations

stations_df = pd.read_csv(os.path.join(data_path, 'stations_5-2.csv'))

In [67]:
# Format locations as latlons

lats = stations_df.latitude
lons = stations_df.longitude

latlons = list(zip(lats.to_list(), lons.to_list()))

In [73]:
# Takes about 1 minute

forecast_df = get_forecast(latlons)

forecast_df

,index,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,probabilityOfPrecipitation,dewpoint,relativeHumidity,windSpeed,windDirection,icon,shortForecast,detailedForecast,pressure
0,0,1,This Afternoon,2023-05-05T12:00:00-05:00,2023-05-05T18:00:00-05:00,True,66,F,None,50,11.111111,68,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,50...",Chance Showers And Thunderstorms,"A chance of rain showers before 1pm, then a ch...",1012.2
1,0,2,Tonight,2023-05-05T18:00:00-05:00,2023-05-06T06:00:00-05:00,False,56,F,None,70,11.666667,90,5 to 10 mph,E,"https://api.weather.gov/icons/land/night/tsra,...",Chance Showers And Thunderstorms then Showers ...,A chance of showers and thunderstorms before 8...,1012.2
2,1,1,This Afternoon,2023-05-05T12:00:00-05:00,2023-05-05T18:00:00-05:00,True,66,F,None,60,10.555556,66,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,60...",Chance Showers And Thunderstorms,"A chance of rain showers before 1pm, then a ch...",1012.2
3,1,2,Tonight,2023-05-05T18:00:00-05:00,2023-05-06T06:00:00-05:00,False,56,F,None,70,11.666667,86,5 to 10 mph,E,"https://api.weather.gov/icons/land/night/tsra,...",Chance Showers And Thunderstorms then Showers ...,A chance of showers and thunderstorms before 8...,1012.2
4,2,1,This Afternoon,2023-05-05T12:00:00-05:00,2023-05-05T18:00:00-05:00,True,66,F,None,50,10.555556,64,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,50...",Chance Showers And Thunderstorms,"A chance of rain showers before 1pm, then a ch...",1012.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,77,2,Tonight,2023-05-05T18:00:00-05:00,2023-05-06T06:00:00-05:00,False,56,F,None,70,11.666667,90,5 to 10 mph,E,"https://api.weather.gov/icons/land/night/tsra,...",Chance Showers And Thunderstorms then Showers ...,A chance of showers and thunderstorms before 8...,1012.2
144,78,1,This Afternoon,2023-05-05T13:00:00-05:00,2023-05-05T18:00:00-05:00,True,65,F,None,50,10.555556,63,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,50...",Chance Showers And Thunderstorms,A chance of showers and thunderstorms before 4...,1012.2
145,78,2,Tonight,2023-05-05T18:00:00-05:00,2023-05-06T06:00:00-05:00,False,55,F,None,70,11.111111,90,5 to 10 mph,E,"https://api.weather.gov/icons/land/night/tsra,...",Chance Showers And Thunderstorms then Showers ...,A chance of showers and thunderstorms before 9...,1012.2
146,79,1,This Afternoon,2023-05-05T13:00:00-05:00,2023-05-05T18:00:00-05:00,True,66,F,None,50,10.555556,63,10 mph,E,"https://api.weather.gov/icons/land/day/tsra,50...",Chance Showers And Thunderstorms,A chance of showers and thunderstorms. Mostly ...,1012.2


### Last Year

In [9]:


# Define the URL for the API endpoint
url = "https://api.weather.gov/stations/{station_id}/observations"

# Replace {station_id} with the station ID for Minneapolis-St. Paul International Airport (ICAO code KMSP)
station_id = "KMSP"

# Define the date range for the observations
start_date = "2022-06-15T00:00:00Z"
end_date = "2023-04-29T23:59:59Z"

# Define the headers for the request
headers = {"Accept": "application/json"}

# Define the query parameters for the request
params = {"start": start_date, "end": end_date}

# Send the request and get the response
response = requests.get(url.format(station_id=station_id), headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()

    # Print the number of observations retrieved
    print("Retrieved {} observations".format(len(data["features"])))
else:
    print("Request failed with status code {}".format(response.status_code))
    print("Error message: {}".format(response.text))

Retrieved 154 observations


### Current

In [4]:
import requests
import json

# Define the URL for the API endpoint
url = "https://api.weather.gov/stations/{station_id}/observations/latest"

# Replace {station_id} with the ID of the station you want to get the data for
station_id = "KMSP"

# Define the headers for the request
headers = {"Accept": "application/geo+json"}

# Send the request and get the response
response = requests.get(url.format(station_id=station_id), headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()

    # Extract the weather information from the data
    temperature = data["properties"]["temperature"]["value"]
    humidity = data["properties"]["relativeHumidity"]["value"]
    wind_speed = data["properties"]["windSpeed"]["value"]
    wind_direction = data["properties"]["windDirection"]["value"]

    # Create a dictionary to store the weather information
    weather_data = {
        "temperature": temperature,
        "humidity": humidity,
        "wind_speed": wind_speed,
        "wind_direction": wind_direction
    }

    # Convert the dictionary to a JSON object
    json_data = json.dumps(weather_data)

    # Print the JSON object to the console
    print(json_data)
else:
    print("Request failed with status code {}".format(response.status_code))

{"temperature": 8.9, "humidity": 53.653768310944, "wind_speed": null, "wind_direction": null}
